In [1]:
import numpy as np
import pandas as pd

admissions = pd.read_csv("../data/ADMISSIONS.csv")
admissions['ADMITTIME'] = pd.to_datetime(admissions['ADMITTIME'])
# load output values
output_values = pd.read_csv("../data/OUTPUTEVENTS.csv",
                            usecols=['HADM_ID', 'ITEMID', 'VALUE', 'CHARTTIME'],
                            low_memory=True)
output_itemids = {
    40055:    "Urine_out_Foley",
    40069:    "Urine_out_Void",
    43175:    "Urine"
}

# === Urine output aggregation for first 24h ===
output_values['CHARTTIME'] = pd.to_datetime(output_values['CHARTTIME'])
mask_out = output_values['ITEMID'].isin(output_itemids.keys())
uro = output_values.loc[mask_out].copy()
uro['measurement'] = uro['ITEMID'].map(output_itemids)
# join ICU admit time
uro = uro.merge(admissions[['HADM_ID','ADMITTIME']], on='HADM_ID', how='left')
uro['hours_since_admit'] = (uro['CHARTTIME'] - uro['ADMITTIME']).dt.total_seconds() / 3600
uro24 = uro[uro['hours_since_admit'] <= 24]
urine_agg = (
    uro24.groupby('HADM_ID')['VALUE']
        .sum()
        .rename('urine_output_24h')
        .reset_index()
)
print(urine_agg.head())
urine_agg.to_csv("output_values.csv", index=False)

    HADM_ID  urine_output_24h
0  100006.0            2950.0
1  100007.0            1105.0
2  100017.0             940.0
3  100029.0               0.0
4  100030.0            1450.0
